In [46]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)

from proyeccion_rdr.produccion.a05_unidades_de_apoyo import guardar_dict_en_excel

ANIOS_POBLACION = [str(i) for i in range(2017, 2036)]

In [48]:
# Lee la poblacion FONASA
ruta_poblaciones = "../data/interim/0.0_poblaciones_ine_y_fonasa_a_utilizar.xlsx"
poblaciones_fonasa = pd.read_excel(ruta_poblaciones, sheet_name="poblacion_FONASA")

# Filtra solamente la poblacion del SSMN y entre 0 y 15 anios (Poblacion que efectivamente
# atendieron). Elimina la columna del 2023, ya que esta mal calculada
poblaciones_fonasa_ssmn = (
    poblaciones_fonasa.query("Estrato == 'SSMN' and `Edad Incidencia` == 'entre_0_y_15'")
    .set_index(["Edad Incidencia", "Estrato"])
    .drop(columns=["2023"])
)
display(poblaciones_fonasa_ssmn)

,,2018,2019,2020,2021,2022
Edad Incidencia,Estrato,,,,,
entre_0_y_15,SSMN,148276,155320,153830,151308,153566


In [49]:
# Lee las consultas de urgencias REM
produccion_urgencias_rem = pd.DataFrame(
    [
        [59, 70, 82, 65, 87, 69, 91, 448, 176, 169],
        [1455, 1292, 2133, 2635, 3096, 1499, 5235, 13613, 12473, 12981],
        [33159, 28148, 25438, 22907, 27428, 13795, 20488, 29809, 28980, 29070],
        [66592, 64954, 63160, 62138, 54425, 21814, 22329, 36763, 37584, 38696],
        [4841, 4781, 0, 0, 1, 0, 0, 0, 0, 0],
        [3764, 3302, 5514, 11914, 8251, 2476, 1484, 933, 1237, 1250],
        [109870, 102547, 96327, 99659, 93288, 39653, 49627, 81566, 80450, 82166],
    ],
    index=["C1", "C2", "C3", "C4", "SIN CATEGORIZACION", "C5", "Total"],
    columns=["2015", "2016", "2017", "2018", "2019", "2020", "2021", "2022", "2023", "2024"],
)
total_urgencias_rem = produccion_urgencias_rem.loc["Total"]

# Obtiene el porcentaje de cada categoria respecto al total
porcentaje_categorizacion_rem = produccion_urgencias_rem / produccion_urgencias_rem.iloc[-1]

In [50]:
# Obtiene la proporcion de urgencias y poblacion
proporcion_urgencias_y_poblacion = total_urgencias_rem / poblaciones_fonasa_ssmn

In [51]:
# Selecciona la cantidad de urgenicas por personas.
# De 15 a 18 fue criterio de experto - Dr Tapia
anio_maximo = proporcion_urgencias_y_poblacion.idxmax(axis=1).iloc[0]
texto_maximo = f"Maximo de urgencias por persona histórico SSMN: {anio_maximo}"
factor_urgencias_0_a_14 = proporcion_urgencias_y_poblacion[anio_maximo].iloc[0]
factor_urgencias_15_a_18 = 1.3

# Extrae los factores de categorizacion para proyectar
factores_categorizacion = porcentaje_categorizacion_rem[anio_maximo].to_frame()
factores_categorizacion["criterio_eleccion"] = texto_maximo
factores_categorizacion = factores_categorizacion.rename(
    columns={anio_maximo: "porcentaje_con_consultas_urg"}
)

# Consolida los factores de urgencias
resumen_factores_elegidos = pd.DataFrame(
    index=["recien_nacidos_vivos", "entre_1_y_14", "entre_15_y_18"],
    columns=["porcentaje_con_consultas_urg", "consultas_por_paciente", "criterio_eleccion"],
    data=[
        [factor_urgencias_0_a_14, 1, texto_maximo],
        [factor_urgencias_0_a_14, 1, texto_maximo],
        [1, factor_urgencias_15_a_18, "Criterio Experto - Dr Tapia"],
    ],
)

# Concatena factores de categorizacion a las variables
resumen_factores_elegidos = pd.concat([resumen_factores_elegidos, factores_categorizacion])

In [52]:
resumen_factores_elegidos

,porcentaje_con_consultas_urg,consultas_por_paciente,criterio_eleccion
recien_nacidos_vivos,0.672118,1.0,Maximo de urgencias por persona histórico SSMN...
entre_1_y_14,0.672118,1.0,Maximo de urgencias por persona histórico SSMN...
entre_15_y_18,1.000000,1.3,Criterio Experto - Dr Tapia
C1,0.000652,NaN,Maximo de urgencias por persona histórico SSMN...
C2,0.026440,NaN,Maximo de urgencias por persona histórico SSMN...
C3,0.229854,NaN,Maximo de urgencias por persona histórico SSMN...
C4,0.623506,NaN,Maximo de urgencias por persona histórico SSMN...
SIN CATEGORIZACION,0.000000,NaN,Maximo de urgencias por persona histórico SSMN...
C5,0.119548,NaN,Maximo de urgencias por persona histórico SSMN...
Total,1.000000,NaN,Maximo de urgencias por persona histórico SSMN...


In [53]:
# Lee la poblacion SSMN extrapolada
estratos_interes_0_a_18 = ["recien_nacidos_vivos", "entre_1_y_14", "entre_15_y_18"]

# Extrae solamente los estratos de interes
poblacion_fonasa_extrapolada_ssmn = (
    pd.read_excel(ruta_poblaciones, sheet_name="poblaciones_fonasa_extrapoladas")
    .query("Estrato == 'SSMN'")
    .set_index("Edad Incidencia")
    .drop(columns="Estrato")
    .query("`Edad Incidencia`.isin(@estratos_interes_0_a_18)")
)


In [54]:
# Multiplica la cantidad de poblacion por el porcentaje que requiere urgencia
pacientes_con_urgencia = poblacion_fonasa_extrapolada_ssmn.mul(
    resumen_factores_elegidos["porcentaje_con_consultas_urg"], axis=0
).dropna()

# Obtiene la cantidad de consultas de urgencia
consultas_urgencia = pacientes_con_urgencia.mul(
    resumen_factores_elegidos["consultas_por_paciente"], axis=0
).dropna()

# Agrega el total de consultas
consultas_urgencia.loc["Total"] = consultas_urgencia.sum()

# Calcula por porcentaje de categorizacion
consultas_categorizacion = pd.DataFrame(
    index=factores_categorizacion.index, data=[consultas_urgencia.loc["Total"]]
)

# Multiplica y calcula las consultas de categorizacion
consultas_categorizacion = (
    consultas_categorizacion.mul(resumen_factores_elegidos["porcentaje_con_consultas_urg"], axis=0)
    .dropna()
    .drop(index="Total")
)

# Resumen final
resumen_consultas = pd.concat([consultas_urgencia, consultas_categorizacion])

In [55]:
print(f"Total consultas urgencia proyectadas 2035: {resumen_consultas.loc['Total', '2035']:.2f}")

Total consultas urgencia proyectadas 2035: 165838.04


In [72]:
resumen_minsal = pd.DataFrame(
    {
        "poblacion_fonasa_2035": poblacion_fonasa_extrapolada_ssmn["2035"],
        "porcentaje_paciente_con_consulta_urgencia": resumen_factores_elegidos[
            "porcentaje_con_consultas_urg"
        ],
        "pacientes_urgencia_2035": pacientes_con_urgencia["2035"],
        "cantidad_consultas_urgencia_por_paciente": resumen_factores_elegidos[
            "consultas_por_paciente"
        ],
        "consultas_urgencia_2035": consultas_urgencia["2035"],
    }
).dropna()

In [73]:
resumen_minsal

,poblacion_fonasa_2035,porcentaje_paciente_con_consulta_urgencia,pacientes_urgencia_2035,cantidad_consultas_urgencia_por_paciente,consultas_urgencia_2035
entre_15_y_18,41844.507000,1.000000,41844.507000,1.3,54397.859100
entre_1_y_14,155392.978665,0.672118,104442.450975,1.0,104442.450975
recien_nacidos_vivos,10411.454388,0.672118,6997.728108,1.0,6997.728108


## Resumen MINSAL

In [56]:
datos_a_guardar = {
    "Proyeccion Urgencias RDR": {
        "poblaciones_FONASA_2018_a_2022": poblaciones_fonasa_ssmn.reset_index(),
        "consultas_REM_urgencia": produccion_urgencias_rem.reset_index(),
        "factores_urgencias_por_pcte_0_a_14": proporcion_urgencias_y_poblacion.reset_index(),
        "factores_urgencias_elegidos": resumen_factores_elegidos.reset_index(),
        "poblaciones_FONASA_extrapoladas": poblacion_fonasa_extrapolada_ssmn_0_a_18.reset_index(),
        "consultas_urgencia_proyectadas": resumen_consultas.reset_index(),
    },
}

NameError: name 'poblacion_fonasa_extrapolada_ssmn_0_a_18' is not defined

In [ ]:
ruta_archivo = "../data/interim/4.0_estimacion_consultas_urgencia.xlsx"
guardar_dict_en_excel(ruta_archivo, datos_a_guardar, espacio_filas=3)